# Introduction: Automated Feature Engineering

In this notebook, we will look at an exciting development in data science: automated feature engineering. A machine learning model can only learn from the data we give it, and making sure that data is relevant to the task is one of the most crucial steps in the machine learning pipeline (this is made clear in the excellent paper ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)). 

However, manual feature engineering is a tedious task and is limited by both human imagination - there are only so many features we can think to create - and by time - creating new features is time-intensive. Ideally, there would be an objective method to create an array of diverse new candidate features that we can then use for a machine learning task. This process is meant to not replace the data scientist, but to make her job easier and allowing her to supplement domain knowledge with an automated workflow.

In this notebook, we will walk through an implementation of using [Featuretools](https://www.featuretools.com/), an open-source Python library for automatically creating features with relational data (where the data is in structured tables). Although there are now many efforts working to enable automated model selection and hyperparameter tuning, there has been a lack of automating work on the feature engineering aspect of the pipeline. This library seeks to close that gap and the general methodology has been proven effective in both [machine learning competitions with the data science machine](https://github.com/HDI-Project/Data-Science-Machine) and [business use cases](https://www.featurelabs.com/blog/predicting-credit-card-fraud/). 


## Dataset

To show the basic idea of featuretools we will use an example dataset consisting of three tables:

* `clients`: information about clients at a credit union
* `loans`: previous loans taken out by the clients
* `payments`: payments made/missed on the previous loans

The general problem of feature engineering is taking disparate data, often distributed across multiple tables, and combining it into a single table that can be used for training a machine learning model. Featuretools has the ability to do this for us, creating many new candidate features with minimal effort. These features are combined into a single table that can then be passed on to our model. 

First, let's load in the data and look at the problem we are working with.

In [134]:
# Run this if featuretools is not already installed
# !pip install -U featuretools

In [135]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [136]:
# Read in the data
clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])

In [137]:
clients.head()

,client_id,joined,income,credit_score
0,39627,2012-12-09,178585,773
1,29029,2005-05-04,168217,510
2,25473,2007-12-08,103032,704
3,40950,2008-07-27,236955,719
4,28543,2003-11-22,65096,639


In [138]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
85,28543,other,8183,1,11275,2006-12-19,2009-06-12,2.26
66,40950,credit,648,1,10737,2010-04-17,2011-10-04,3.58
195,41806,other,6535,0,10198,2007-02-25,2008-08-11,7.39
135,32291,cash,1619,1,11961,2014-10-08,2017-05-03,2.51
183,34263,home,6020,1,11453,2005-05-23,2007-05-14,5.47
128,47687,home,5253,0,10859,2009-09-23,2011-12-19,4.15
26,29029,cash,7334,0,11189,2005-04-28,2007-06-01,0.53
18,39627,home,1973,1,10315,2011-04-17,2013-04-17,0.76
155,35042,cash,5293,0,11436,2006-04-13,2008-05-19,2.53
167,35042,credit,5005,0,11537,2001-12-06,2003-05-01,5.71


In [139]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
2963,11835,1249,2006-05-02,1
3103,11685,1042,2003-04-24,0
622,11249,1676,2000-07-17,0
3353,11130,1313,2007-11-16,1
1700,10483,1030,2000-10-20,1
2888,11491,1386,2003-05-13,0
2041,11779,1048,2010-10-24,0
451,10888,1289,2005-12-17,0
2059,11255,1498,2004-05-06,1
1557,10597,1558,2002-09-08,1


### Manual Feature Engineering Examples

Let's show a few examples of features we might make by hand. We will keep this relatively simple to avoid doing too much work! First we will focus on a single dataframe before combining them together. In the `clients` dataframe, we can take the month of the `joined` column and the natural log of the `income` column. Later, we see these are known in featuretools as transformation feature primitives because they act on column in a single table. 

In [140]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,39627,2012-12-09,178585,773,12,12.092820
1,29029,2005-05-04,168217,510,5,12.033010
2,25473,2007-12-08,103032,704,12,11.542795
3,40950,2008-07-27,236955,719,7,12.375626
4,28543,2003-11-22,65096,639,11,11.083618


To incorporate information about the other tables, we use the `df.groupby` method, followed by a suitable aggregation function, followed by `df.merge`.  For example, let's calculate the average, minimum, and maximum amount of previous loans for each client. In the terms of featuretools, this would be considered an aggregation feature primitive because we using multiple tables in a one-to-many relationship to calculate aggregation figures (don't worry, this will be explained shortly!).

In [141]:
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.head()

,mean,max,min
client_id,,,
25473,6895.000000,13472,650
25798,9905.117647,14763,1274
26053,11068.111111,14969,2516
28543,7682.894737,13339,1003
28794,7305.823529,14470,1515


In [142]:
# Groupby client id and calculate mean, max, min previous loan size
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25473,6895.000000,13472,650
25798,9905.117647,14763,1274
26053,11068.111111,14969,2516
28543,7682.894737,13339,1003
28794,7305.823529,14470,1515


In [143]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,39627,2012-12-09,178585,773,12,12.092820,7077.250000,13427,1973
1,29029,2005-05-04,168217,510,5,12.033010,7665.000000,14617,640
2,25473,2007-12-08,103032,704,12,11.542795,6895.000000,13472,650
3,40950,2008-07-27,236955,719,7,12.375626,7607.789474,13666,648
4,28543,2003-11-22,65096,639,11,11.083618,7682.894737,13339,1003
5,47487,2001-01-08,34377,543,1,10.445143,7249.117647,13042,938
6,47687,2002-03-13,194871,779,3,12.180093,6121.473684,13757,685
7,32291,2010-01-04,189446,810,1,12.151859,7820.842105,14454,1001
8,35042,2004-03-08,105286,592,3,11.564436,7836.700000,14750,1130
9,34263,2000-12-19,183226,827,12,12.118476,8169.470588,14261,3809


We could go further and include information about `payments` in the `clients` dataframe. To do so, we would have to group `payments` by the `loan_id`, merge it with the `loans`, group the resulting dataframe by the `client_id`, and then merge it into the `clients` dataframe. This would allow us to include information about previous payments for each client. 

Clearly, this process of manual feature engineering can grow quite tedious with many columns and multiple tables and I certainly don't want to have to do this process by hand! Luckily, featuretools can automatically perform this entire process and will create more features than we would have ever thought of. Although I love `pandas`, there is only so much manual data manipulation I'm willing to stand! 

# Featuretools

Now that we know what we are trying to avoid (tedious manual feature engineering), let's figure out how to automate this process. Featuretools operates on an idea known as [Deep Feature Synthesis](https://docs.featuretools.com/api_reference.html#deep-feature-synthesis). You can read the [original paper here](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf), and although it's quite readable, it's not necessary to understand the details to do automated feature engineering. The concept of Deep Feature Synthesis is to use basic building blocks known as feature primitives (like the transformations and aggregations done above) that can be stacked on top of each other to form new features. The depth of a "deep feature" is equal to the number of stacked primitives. 

I threw out some terms there, but don't worry because we'll cover them as we go. Featuretools builds on simple ideas to create a powerful method, and we will build up our understanding in much the same way. 

The first part of Featuretools to understand [is an `entity`](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-entities). This is simply a table, or in `pandas`, a `DataFrame`. We corral multiple entities into a [single object called an `EntitySet`](https://docs.featuretools.com/loading_data/using_entitysets.html). This is just a large data structure composed of many individual entities and the relationships between them.  

## EntitySet

Creating a new `EntitySet` is pretty simple: 

In [144]:
es = ft.EntitySet(id = 'clients')
es

Entityset: clients
  DataFrames:
  Relationships:
    No relationships

## Entities 

An entity is simply a table, which is represented in Pandas as a `dataframe`. Each entity must have a uniquely identifying column, known as an index. For the clients dataframe, this is the `client_id` because each id only appears once in the `clients` data. In the `loans` dataframe, `client_id` is not an index because each id might appear more than once. The index for this dataframe is instead `loan_id`. 

When we create an `entity` in featuretools, we have to identify which column of the dataframe is the index. If the data does not have a unique index we can tell featuretools to make an index for the entity by passing in `make_index = True` and specifying a name for the index. If the data also has a uniquely identifying time index, we can pass that in as the `time_index` parameter. 

Featuretools will automatically infer the variable types (numeric, categorical, datetime) of the columns in our data, but we can also pass in specific datatypes to override this behavior. As an example, even though the `repaid` column in the `loans` dataframe is represented as an integer, we can tell featuretools that this is a categorical feature since it can only take on two discrete values. This is done using an integer with the variables as keys and the feature types as values.

In the code below we create the three entities and add them to the `EntitySet`.  The syntax is relatively straightforward with a few notes: for the `payments` dataframe we need to make an index, for the `loans` dataframe, we specify that `repaid` is a categorical variable, and for the `payments` dataframe, we specify that `missed` is a categorical feature. 

In [145]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.add_dataframe(dataframe_name = 'clients', 
                      dataframe = clients, 
                      index = 'client_id',
                      time_index = 'joined')
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
  Relationships:
    No relationships

In [146]:
from woodwork.logical_types import Categorical, PostalCode
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.add_dataframe(dataframe_name = 'loans', 
                      dataframe = loans, 
                      logical_types = {'repaid': Categorical},
                      index = 'loan_id', 
                      time_index = 'loan_start')
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
  Relationships:
    No relationships

In [147]:
from woodwork.logical_types import Categorical, PostalCode
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.add_dataframe(dataframe_name = 'payments', 
                      dataframe = payments,
                      logical_types = {'missed': Categorical},
                      make_index = True,
                      index = 'payment_id',
                      time_index = 'payment_date')
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3474, Columns: 5]
  Relationships:
    No relationships

All three entities have been successfully added to the `EntitySet`. We can access any of the entities using Python dictionary syntax.

In [148]:
es['loans'].head()

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
10707,40950,cash,2994,0,10707,2000-01-22,2002-05-13,0.39
11151,43302,cash,5464,0,11151,2000-01-25,2001-11-12,1.36
10380,37403,home,3712,1,10380,2000-01-26,2001-10-20,4.41
11249,28543,credit,8976,0,11249,2000-02-03,2002-03-15,5.31
10882,46342,home,6464,0,10882,2000-02-04,2001-06-28,1.33


Featuretools correctly inferred each of the datatypes when we made this entity. We can also see that we overrode the type for the `repaid` feature, changing if from numeric to categorical. 

In [149]:
es["clients"].head()

,client_id,joined,income,credit_score,join_month,log_income
41806,41806,2000-06-27,179340,502,6,12.097039
34263,34263,2000-12-19,183226,827,12,12.118476
47487,47487,2001-01-08,34377,543,1,10.445143
47687,47687,2002-03-13,194871,779,3,12.180093
43049,43049,2003-03-13,80535,513,3,11.296447


In [150]:
es['payments'].head()

,payment_id,loan_id,payment_amount,payment_date,missed
1885,1885,11151,886,2000-03-09,1
961,961,10104,1291,2000-03-26,1
3390,3390,10882,1076,2000-03-26,1
1886,1886,11151,785,2000-04-01,0
498,498,10707,522,2000-04-07,0


## Relationships

After defining the entities (tables) in an `EntitySet`, we now need to tell featuretools [how they are related with a relationship](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-a-relationship). The most intuitive way to think of relationships is with the parent to child analogy: a parent-to-child relationship is one-to-many because for each parent, there can be multiple children. The `client` dataframe is therefore the parent of the `loans` dataframe because while there is only one row for each client in the `client` dataframe, each client may have several previous loans covering multiple rows in the `loans` dataframe. Likewise, the `loans` dataframe is the parent of the `payments` dataframe because each loan will have multiple payments. 

These relationships are what allow us to group together datapoints using aggregation primitives and then create new features. As an example, we can group all of the previous loans associated with one client and find the average loan amount. We will discuss the features themselves more in a little bit, but for now let's define the relationships. 

To define relationships, we need to specify the parent variable and the child variable. This is the variable that links two entities together. In our example, the `client` and `loans` dataframes are linked together by the `client_id` column. Again, this is a parent to child relationship because for each `client_id` in the parent `client` dataframe, there may be multiple entries of the same `client_id` in the child `loans` dataframe. 

We codify relationships in the language of featuretools by specifying the parent variable and then the child variable. After creating a relationship, we add it to the `EntitySet`. 

In [151]:
print(ft.__version__)

1.9.2


In [152]:
# Relationship between clients and previous loans
relationships = [("clients", "client_id", "loans", "client_id"),
                 ("loans", "loan_id", "payments", "loan_id")]
es.add_relationships(relationships)

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3474, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

We now have our entities in an entityset along with the relationships between them. We can now start to making new features from all of the tables using stacks of feature primitives to form deep features. First, let's cover feature primitives.


## Feature Primitives

A [feature primitive](https://docs.featuretools.com/automated_feature_engineering/primitives.html) a at a very high-level is an operation applied to data to create a feature. These represent very simple calculations that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

* __Aggregation__: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum loan amount for each client. An aggregation works across multiple tables using relationships between tables.
* __Transformation__: an operation applied to one or more columns in a single table. An example would be extracting the day from dates, or finding the difference between two columns in one table.

Let's take a look at feature primitives in featuretools. We can view the list of primitives:

In [153]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 20
primitives[primitives['type'] == 'aggregation']

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,percent_true,aggregation,True,False,Determines the p...,<ColumnSchema (L...,None
1,entropy,aggregation,False,False,Calculates the e...,<ColumnSchema (S...,None
2,std,aggregation,True,True,Computes the dis...,<ColumnSchema (S...,None
3,median,aggregation,False,False,Determines the m...,<ColumnSchema (S...,None
4,avg_time_between,aggregation,False,False,Computes the ave...,<ColumnSchema (L...,None
5,sum,aggregation,True,True,Calculates the t...,<ColumnSchema (S...,None
6,num_true,aggregation,True,False,Counts the numbe...,<ColumnSchema (L...,None
7,trend,aggregation,False,False,Calculates the t...,<ColumnSchema (L...,None
8,max,aggregation,True,True,Calculates the h...,<ColumnSchema (S...,None
9,time_since_last,aggregation,False,False,Calculates the t...,<ColumnSchema (L...,None


In [154]:
transforms = primitives[primitives['type'] == 'transform']
transforms.head(20)
# transforms[transforms.name == "month"]

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
22,absolute,transform,True,True,Computes the abs...,<ColumnSchema (S...,None
23,less_than_equal_...,transform,True,True,Determines if va...,<ColumnSchema (L...,None
24,is_in_geobox,transform,False,False,Determines if co...,<ColumnSchema (L...,None
25,email_address_to...,transform,False,False,Determines the d...,<ColumnSchema (L...,None
26,diff,transform,False,False,Compute the diff...,<ColumnSchema (S...,None
27,add_numeric_scalar,transform,True,True,Add a scalar to ...,<ColumnSchema (S...,None
28,modulo_by_feature,transform,True,True,Return the modul...,<ColumnSchema (S...,None
29,is_federal_holiday,transform,False,False,Determines if a ...,<ColumnSchema (L...,None
30,month,transform,True,True,Determines the m...,<ColumnSchema (L...,None
31,square_root,transform,True,True,Computes the squ...,<ColumnSchema (S...,None


If featuretools does not have enough primitives for us, we can [also make our own.](https://docs.featuretools.com/automated_feature_engineering/primitives.html#defining-custom-primitives) 

To get an idea of what a feature primitive actually does, let's try out a few on our data. Using primitives is surprisingly easy using the `ft.dfs` function (which stands for deep feature synthesis). In this function, we specify the entityset to use; the `target_entity`, which is the dataframe we want to make the features for (where the features end up); the `agg_primitives` which are the aggregation feature primitives; and the `trans_primitives` which are the transformation primitives to apply. 

In the following example, we are using the `EntitySet` we already created, the target entity is the `clients` dataframe because we want to make new features about each client, and then we specify a few aggregation and transformation primitives. 

In [155]:
# Create new features using specified primitives
features, feature_matrix = ft.dfs(entityset = es, target_dataframe_name = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['year', 'month', 'diff'])
features

,income,credit_score,join_month,log_income,LAST(loans.loan_amount),LAST(loans.loan_id),LAST(loans.loan_type),LAST(loans.rate),LAST(loans.repaid),MAX(loans.loan_amount),...,DIFF(MAX(payments.payment_amount)),DIFF(MEAN(loans.loan_amount)),DIFF(MEAN(loans.rate)),DIFF(MEAN(payments.payment_amount)),MONTH(LAST(loans.loan_end)),MONTH(LAST(loans.loan_start)),MONTH(LAST(payments.payment_date)),YEAR(LAST(loans.loan_end)),YEAR(LAST(loans.loan_start)),YEAR(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
41806,179340,502,6,12.097039,10483,10047,other,3.03,1,13230.0,...,NaN,NaN,NaN,NaN,2,10,10,2016,2013,2014
34263,183226,827,12,12.118476,3809,11312,credit,0.13,1,14261.0,...,434.0,-863.362745,-0.433725,-153.619606,11,10,7,2016,2014,2015
47487,34377,543,1,10.445143,5090,11267,credit,2.76,0,13042.0,...,-355.0,-920.352941,0.524706,-170.610577,3,5,12,2016,2014,2014
47687,194871,779,3,12.180093,685,11219,credit,6.09,0,13757.0,...,200.0,-1127.643963,-0.828700,-60.897998,5,9,4,2016,2014,2015
43049,80535,513,3,11.296447,542,11131,credit,2.62,1,13719.0,...,26.0,28.291022,-0.366006,1.168831,12,12,8,2016,2014,2015
28543,65096,639,11,11.083618,2649,11973,cash,0.94,1,13339.0,...,-59.0,1533.130031,1.013901,297.094675,8,9,6,2016,2014,2015
35042,105286,592,3,11.564436,14750,11170,home,2.79,1,14750.0,...,233.0,153.805263,-0.943342,-100.154198,11,3,12,2015,2013,2013
28794,155095,684,1,11.951793,1515,10880,credit,0.33,0,14470.0,...,-65.0,-530.876471,0.680618,-2.861529,1,10,4,2016,2013,2014
29029,168217,510,5,12.033010,7285,11662,credit,0.45,1,14617.0,...,-75.0,359.176471,-0.296118,-110.395332,2,12,12,2016,2013,2014


In [156]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
41806,6
34263,12
47487,1
47687,3
43049,3


In [157]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
41806,1353.959350
34263,1200.339744
47487,1029.729167
47687,968.831169
43049,970.000000


In [158]:
features.head()

,income,credit_score,join_month,log_income,LAST(loans.loan_amount),LAST(loans.loan_id),LAST(loans.loan_type),LAST(loans.rate),LAST(loans.repaid),MAX(loans.loan_amount),...,DIFF(MAX(payments.payment_amount)),DIFF(MEAN(loans.loan_amount)),DIFF(MEAN(loans.rate)),DIFF(MEAN(payments.payment_amount)),MONTH(LAST(loans.loan_end)),MONTH(LAST(loans.loan_start)),MONTH(LAST(payments.payment_date)),YEAR(LAST(loans.loan_end)),YEAR(LAST(loans.loan_start)),YEAR(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
41806,179340,502,6,12.097039,10483,10047,other,3.03,1,13230.0,...,NaN,NaN,NaN,NaN,2,10,10,2016,2013,2014
34263,183226,827,12,12.118476,3809,11312,credit,0.13,1,14261.0,...,434.0,-863.362745,-0.433725,-153.619606,11,10,7,2016,2014,2015
47487,34377,543,1,10.445143,5090,11267,credit,2.76,0,13042.0,...,-355.0,-920.352941,0.524706,-170.610577,3,5,12,2016,2014,2014
47687,194871,779,3,12.180093,685,11219,credit,6.09,0,13757.0,...,200.0,-1127.643963,-0.828700,-60.897998,5,9,4,2016,2014,2015
43049,80535,513,3,11.296447,542,11131,credit,2.62,1,13719.0,...,26.0,28.291022,-0.366006,1.168831,12,12,8,2016,2014,2015


Already we can see how useful featuretools is: it performed the same operations we did manually but also many more in addition. Examining the names of the features in the dataframe brings us to the final piece of the puzzle: deep features.

## Deep Feature Synthesis

While feature primitives are useful by themselves, the main benefit of using featuretools arises when we stack primitives to get deep features. The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1, a feature that stacks two primitives would have a depth of 2 and so on. The idea itself is lot simpler than the name "deep feature synthesis" implies. (I think the authors were trying to ride the way of deep neural network hype when they named the method!) To read more about deep feature synthesis, check out [the documentation](https://docs.featuretools.com/automated_feature_engineering/afe.html) or the [original paper by Max Kanter et al](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf). 

Already in the dataframe we made by specifying the primitives manually we can see the idea of feature depth. For instance, the MEAN(loans.loan_amount) feature has a depth of 1 because it is made by applying a single aggregation primitive. This feature represents the average size of a client's previous loans.

In [159]:
# Show a feature with a depth of 1
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

,MEAN(loans.loan_amount)
client_id,
41806,9032.833333
34263,8169.470588
47487,7249.117647
47687,6121.473684
43049,6149.764706
28543,7682.894737
35042,7836.700000
28794,7305.823529
29029,7665.000000


As well scroll through the features, we see a number of features with a depth of 2. For example, the LAST(loans.(MEAN(payments.payment_amount))) has depth = 2 because it is made by stacking two feature primitives, first an aggregation and then a transformation. This feature represents the average payment amount for the last (most 
recent) loan for each client.

In [160]:
# Show a feature with a depth of 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
41806,1611.111111
34263,869.285714
47487,811.250000
47687,111.500000
43049,330.916667
28543,399.625000
35042,1874.214286
28794,253.600000
29029,1096.400000


We can create features of arbitrary depth by stacking more primitives. However, when I have used featuretools I've never gone beyond a depth of 2! After this point, the features become very convoluted to understand. I'd encourage anyone interested to experiment with increasing the depth (maybe for a real problem) and see if there is value to "going deeper".

## Automated Deep Feature Synthesis

In addition to manually specifying aggregation and transformation feature primitives, we can let featuretools automatically generate many new features. We do this by making the same `ft.dfs` function call, but without passing in any primitives. We just set the `max_depth` parameter and featuretools will automatically try many all combinations of feature primitives to the ordered depth. 

When running on large datasets, this process can take quite a while, but for our example data, it will be relatively quick. For this call, we only need to specify the `entityset`, the `target_entity` (which will again be `clients`), and the `max_depth`. 

In [166]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, 
                                 target_dataframe_name='clients', 
                                 max_depth = 2)
features.shape

(25, 110)

In [167]:
features.iloc[:, 4:].head()

,COUNT(loans),MAX(loans.loan_amount),MAX(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),MODE(loans.loan_type),MODE(loans.repaid),NUM_UNIQUE(loans.loan_type),...,MODE(payments.loans.loan_type),MODE(payments.loans.repaid),NUM_UNIQUE(payments.loans.loan_type),NUM_UNIQUE(payments.loans.repaid),SKEW(payments.loans.loan_amount),SKEW(payments.loans.rate),STD(payments.loans.loan_amount),STD(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
41806,18,13230.0,7.95,9032.833333,3.796667,2167.0,0.11,home,0,4,...,home,0,4,2,-0.652701,0.382510,3050.737698,2.024768,1113912.0,476.45
34263,17,14261.0,10.71,8169.470588,3.362941,3809.0,0.13,home,0,4,...,home,0,4,2,0.250499,1.345623,3187.443050,2.823355,1282800.0,476.60
47487,17,13042.0,6.94,7249.117647,3.887647,938.0,0.55,credit,1,4,...,other,1,4,2,-0.166832,0.051584,3980.896814,1.441264,1076247.0,555.41
47687,19,13757.0,8.43,6121.473684,3.058947,685.0,0.48,credit,0,4,...,credit,0,4,2,0.120829,0.702181,4120.198858,2.171590,970526.0,486.94
43049,17,13719.0,7.58,6149.764706,2.692941,542.0,0.47,credit,1,4,...,credit,1,4,2,0.347559,1.115351,3818.469690,1.771155,733733.0,318.38


Deep feature synthesis has created 90 new features out of the existing data! While we could have created all of these manually, I am glad to not have to write all that code by hand. The primary benefit of featuretools is that it creates features without any subjective human biases. Even a human with considerable domain knowledge will be limited by their imagination when making new features (not to mention time). Automated feature engineering is not limited by these factors (instead it's limited by computation time) and provides a good starting point for feature creation. This process likely will not remove the human contribution to feature engineering completely because a human can still use domain knowledge and machine learning expertise to select the most important features or build new features from those suggested by automated deep feature synthesis.

# Next Steps

While automatic feature engineering solves one problem, it provides us with another problem: too many features! Although it's difficult to say which features will be important to a given machine learning task ahead of time, it's likely that not all of the features made by featuretools add value. In fact, having too many features is a significant issue in machine learning because it makes training a model much harder. The [irrelevant features can drown out the important features](https://pdfs.semanticscholar.org/a83b/ddb34618cc68f1014ca12eef7f537825d104.pdf), leaving a model unable to learn how to map the features to the target.

This problem is known as the ["curse of dimensionality"](https://en.wikipedia.org/wiki/Curse_of_dimensionality#Machine_learning) and is addressed through the process of [feature reduction and selection](http://scikit-learn.org/stable/modules/feature_selection.html), which means [removing low-value features](https://machinelearningmastery.com/feature-selection-machine-learning-python/) from the data. Defining which features are useful is an important problem where a data scientist can still add considerable value to the feature engineering task. Feature reduction will have to be another topic for another day!

------
Feature selection
https://scikit-learn.org/stable/modules/feature_selection.html
!!!!
_____

# Conclusions

In this notebook, we saw how to apply automated feature engineering to an example dataset. This is a powerful method which allows us to overcome the human limits of time and imagination to create many new features from multiple tables of data. Featuretools is built on the idea of deep feature synthesis, which means stacking multiple simple feature primitives - __aggregations and transformations__ - to create new features. Feature engineering allows us to combine information across many tables into a single dataframe that we can then use for machine learning model training. Finally, the next step after creating all of these features is figuring out which ones are important. 

Featuretools is currently the only Python option for this process, but with the recent emphasis on automating aspects of the machine learning pipeline, other competitiors will probably enter the sphere. While the exact tools will change, the idea of automatically creating new features out of existing data will grow in importance. Staying up-to-date on methods such as automated feature engineering is crucial in the rapidly changing field of data science. Now go out there and find a problem on which to apply featuretools! 

For more information, check out the [documentation for featuretools](https://docs.featuretools.com/index.html). Also, read about how featuretools is [used in the real world by Feature Labs](https://www.featurelabs.com/), the company behind the open-source library.